# *Fonctions de calcul des critères de dynamique et de création de datasets utilisables pour les PA Energy Yield (EY) et Gross Margin (GM)*

In [ ]:
# Importation librairies 
library(tidyverse)
library(reshape2)
library(cowplot)
library(magrittr)
library(ggplot2)
library(corrplot)
library(slider)
library(nlme)

In [2]:
# Création des sous fichiers à partir des données originelles 
datafarm <- read.csv2("PAs_farm_df.csv", dec=".", sep=";", header=T)
datater <- read.csv2("PAs_terr_df.csv", dec=".", sep=";", header=T)
dataEYfarm <- subset(datafarm, select = c(1, 2, 3, 4, 5, 6, 7, 8))
dataGMfarm <- subset(datafarm, select = c(1, 2, 3, 4, 5, 6, 7, 10))
dataEYter <- subset(datater, select = c(1, 2, 3, 4, 5, 6))
dataGMter <- subset(datater, select = c(1, 2, 3, 4, 5, 8))

### 1. Fonctions de calcul de la métrique MEAN pour le CD Level

#### 1.1_ Au niveau Farm et Group

In [12]:
levelfarm <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(levelFarm = mean(perf))%>%# mean level
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(levelGroup = mean(MoyGroup))%>%
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".level_Farm"),paste0(name,".level_Group"))
    colnames(df)[i] <- name
  }
  return(df)
}

#### 1.2_ Au niveau Territoire

In [13]:
levelter <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    #df <- data
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(levelTerritory = mean(MoyTer))%>%
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".level_Territory")
    colnames(df)[i] <- name
  }
  return(df)
}

### 2. Fonctions de calcul de la métrique SLOPE OF LINEAR REGRESSION pour le CD Trend

#### 2.1_ Au niveau Farm et Group

In [14]:
trendfarm <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(TRENDf_Farm = if(all(is.na(perf))) NA else lm(perf ~ Year, na.action = na.exclude)$coefficients[[2]])%>% # slope of linear regression
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(TRENDf_Group = if(all(is.na(MoyGroup))) NA else lm(MoyGroup ~ Year, na.action = na.exclude)$coefficients[[2]])%>% # slope of linear regression
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".TRENDf_Farm"),paste0(name,".TRENDf_Group"))
    colnames(df)[i] <- name
  }
  return(df)
  
}

#### 2.2_ Au niveau Territoire

In [15]:
trendter <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(TRENDf_Territory = if(all(is.na(MoyTer))) NA else lm(MoyTer ~ Year, na.action = na.exclude)$coefficients[[2]])%>% # slope of linear regression
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".TRENDf_Territory")
    colnames(df)[i] <- name
  }
  return(df)
  
}

### 3. Fonctions de calcul de la métrique RSD pour le CD Variability

#### 3.1_ Au niveau Farm et Group

In [16]:
variabilityfarm_RSD <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(RSD_Farm = abs(sd(perf)/mean(perf))*100)%>% # relative standard deviation 
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(RSD_Group = abs(sd(MoyGroup)/mean(MoyGroup))*100)%>% # relative standard deviation 
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".RSD_Farm"),paste0(name,".RSD_Group"))
    colnames(df)[i] <- name
  }  
  return(df)
} 

#### 3.2_ Au niveau Territoire

In [17]:
variabilityter_RSD <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(RSD_Territory = abs(sd(MoyTer)/mean(MoyTer))*100)%>% # relative standard deviation 
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".RSD_Territory")
    colnames(df)[i] <- name
  }  
  return(df)
} 

### 4. Fonctions de calcul de la métrique SSR pour le CD Variability

#### 4.1_ Au niveau Farm et Group

In [18]:
variabilityfarm_SSR <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(SSR_Farm = sum((predict(lm(perf ~ Year, na.action = na.exclude)) - mean(perf))^2))%>% # sum of squared residuals 
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(SSR_Group = sum((predict(lm(MoyGroup ~ Year, na.action = na.exclude)) - mean(MoyGroup))^2))%>% # sum of squared residuals
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".SSR_Farm"),paste0(name,".SSR_Group"))
    colnames(df)[i] <- name
  }  
  return(df)
}   

#### 4.2_ Au niveau Territoire

In [19]:
variabilityter_SSR <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(SSR_Territory = sum((predict(lm(MoyTer ~ Year, na.action = na.exclude)) - mean(MoyTer))^2))%>% # sum of squared residuals
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".SSR_Territory")
    colnames(df)[i] <- name
  }  
  return(df)
}   

### 5. Fonctions de calcul de la métrique NOD pour le CD Resistance

#### 5.1_ Au niveau Farm et Group

In [20]:
resistancefarm_disr <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(NOD_Farm = length(perf[perf < 0.75*slide_dbl(perf, mean, .before = 2)]))%>% # number of disruption 
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(NOD_Group = length(MoyGroup[MoyGroup < 0.75*slide_dbl(MoyGroup, mean, na.action = na.omit, .before = 2)]))%>% # number of disruption
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".NOD_Farm"),paste0(name,".NOD_Group"))
    colnames(df)[i] <- name
  }  
  return(df)
  
}

#### 5.2_ Au niveau Territoire

In [21]:
resistanceter_disr <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(NOD_Territory = length(MoyTer[MoyTer < 0.75*slide_dbl(MoyTer, mean, na.action = na.omit, .before = 2)]))%>% # number of disruption 
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".NOD_Territory")
    colnames(df)[i] <- name
  }  
  return(df)
  
}

### 6. Fonctions de calcul de la métrique Probahigh pour le CD Resistance

#### 6.1_ Au niveau Farm et Group

In [22]:
resistancefarm_probapeak <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 8:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df%>%
      group_by(Farm, RCP, SSP)%>%
      mutate(probahigh_Farm = pnorm((1.25*mean(perf) - mean(perf))/sd(perf)))%>%# probability of high performance level with threshold adapted by individuals Macholdt et al 2020
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP, Year)%>%
      mutate(MoyGroup = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(Farm_group, RCP, SSP)%>%
      mutate(probahigh_Group  = pnorm((1.25*mean(MoyGroup) - mean(MoyGroup))/sd(MoyGroup)))%>%# probability of high performance level with threshold adapted by individuals Macholdt et al 2020
      ungroup()
    
    df <- select(df, - MoyGroup)
    
    colnames(df)[c((ncol(df)-1): ncol(df))] <- c(paste0(name,".probahigh_Farm"),paste0(name,".probahigh_Group"))
    colnames(df)[i] <- name
  }  
  return(df)
  
}

#### 6.2_ Au niveau Territoire

In [23]:
resistanceter_probapeak <- function(data, nperf, mod){
  
  df <- subset(data, Mod == mod)
  
  for(i in 6:nperf){
    
    name <- colnames(df)[i]
    
    colnames(df)[i] <- "perf"
    
    df <- df %>%
      group_by(RCP, SSP, Year)%>%
      mutate(MoyTer = mean(perf))%>%
      ungroup()
    
    df <- df %>%
      group_by(RCP, SSP)%>%
      mutate(probahigh_Territory = pnorm((1.25*mean(MoyTer) - mean(MoyTer))/sd(MoyTer)))%>%# probability of high performance level with threshold adapted by individuals Macholdt et al 2020
      ungroup()
    
    df <- select(df, - MoyTer)
    
    colnames(df)[ncol(df)] <- paste0(name,".probahigh_Territory")
    colnames(df)[i] <- name
  }  
  return(df)
  
}

### 7. Fonctions de création d'un dataset augmenté du calcul de toutes les métriques pour tous les levels et les PA donnés

#### 7.1_ Au niveau Farm et Group

In [24]:
criteriadynamics_farm <- function(datafarm, mod, nomfichierfarm){
  
  data_levelfarm <- levelfarm(datafarm, ncol(datafarm), mod)
  data_trendfarm <- trendfarm(data_levelfarm, ncol(datafarm), mod)
  data_varRSDfarm <- variabilityfarm_RSD(data_trendfarm, ncol(datafarm), mod)
  data_varSSRfarm <- variabilityfarm_SSR(data_varRSDfarm, ncol(datafarm), mod)
  data_resdisrfarm <- resistancefarm_disr(data_varSSRfarm, ncol(datafarm), mod)
  data_respeakfarm <- resistancefarm_probapeak(data_resdisrfarm, ncol(datafarm), mod)
  
  #Save file
  #write_excel_csv(data_respeakfarm, paste0(nomfichierfarm, mod, "Criteria_Data.csv"), delim = ";", na = "")
  write.table(data_respeakfarm, paste0(nomfichierfarm, mod, "Criteria_Data.csv"), append = FALSE, sep = ";", dec = ".", row.names = FALSE)
  
  return(data_respeakfarm)
}

#### 7.2_ Au niveau Territoire

In [25]:
criteriadynamics_ter <- function(datater, mod, nomfichierter){
  
  data_levelter <- levelter(datater, ncol(datater), mod)
  data_trendter <- trendter(data_levelter, ncol(datater), mod)
  data_varRSDter <- variabilityter_RSD(data_trendter, ncol(datater), mod)
  data_varSSRter <- variabilityter_SSR(data_varRSDter, ncol(datater), mod)
  data_resdisrter <- resistanceter_disr(data_varSSRter, ncol(datater), mod)
  data_respeakter <- resistanceter_probapeak(data_resdisrter, ncol(datater), mod)
  
  #Save file
  #write_excel_csv(data_respeakter, paste0(nomfichierter, mod, "Criteria_Data.csv"), delim = ";", na = "")
  write.table(data_respeakter, paste0(nomfichierter, mod, "Criteria_Data.csv"), append = FALSE, sep = ";", dec = ".", row.names = FALSE)
  
  return(data_respeakter)
}